In [8]:
from langgraph.graph import StateGraph, START, END
from langchain.chat_models import init_chat_model
gemini_llm = init_chat_model(model="gemini-2.0-flash-lite-001",model_provider="google_vertexai")
gemini_llm.invoke("where is Apply headquarters located")


c:\Users\srina\OneDrive\Desktop\GenAI_code\ConditionalGraph\.venv\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
c:\Users\srina\OneDrive\Desktop\GenAI_code\ConditionalGraph\.venv\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


AIMessage(content="Apple's headquarters, often referred to as Apple Park, is located in **Cupertino, California, USA**.\n", additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 5, 'candidates_token_count': 24, 'total_token_count': 29, 'prompt_tokens_details': [{'modality': 1, 'token_count': 5}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 24}], 'thoughts_token_count': 0, 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.20599464575449625, 'model_name': 'gemini-2.0-flash-lite-001'}, id='run--50d3d00f-be28-467d-8500-03452bc35df9-0', usage_metadata={'input_tokens': 5, 'output_tokens': 24, 'total_tokens': 29})

In [1]:
from langgraph.graph import MessagesState
from langchain.tools import tool

In [3]:
@tool("addition", parse_docstring=True, return_direct=True)
def add(a: int, b: int) -> int:
    """Addition of two numbers

    Args:
      a : number
      b : number

    """
    return a + b

In [4]:
@tool("multipy", parse_docstring=True, return_direct=True)
def mul(a: int, b: int) -> int:
    """Multiplication of two numbers

    Args:
      a : number
      b : number

    """
    return a * b

In [6]:
@tool("divide", parse_docstring=True, return_direct=True)
def div(a: int, b: int) -> int:
    """division of two numbers

    Args:
      a : number
      b : number

    """
    return a // b

In [2]:
@tool("subtraction", parse_docstring=True, return_direct=True)
def sub(a: int, b: int) -> int:
    """Subtract two numbers

    Args:
      a : number
      b : number

    """
    return a - b

In [18]:
gemini_llm_with_tools = gemini_llm.bind_tools([add])

In [19]:
result = gemini_llm_with_tools.invoke("what is 2 plus 2?")
result

AIMessage(content='', additional_kwargs={'function_call': {'name': 'add', 'arguments': '{"a": 2.0, "b": 2.0}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 34, 'candidates_token_count': 5, 'total_token_count': 39, 'prompt_tokens_details': [{'modality': 1, 'token_count': 34}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 5}], 'thoughts_token_count': 0, 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.007441814243793488, 'model_name': 'gemini-2.0-flash-lite-001'}, id='run--ec907868-25cf-4ddc-b0d2-1fb1ee83d47b-0', tool_calls=[{'name': 'add', 'args': {'a': 2.0, 'b': 2.0}, 'id': '648ad77b-f4cf-4914-ba40-6b5cffdba98d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 34, 'output_tokens': 5, 'total_tokens': 39})

In [20]:
result = gemini_llm_with_tools.invoke("what is capital of india?")
result

AIMessage(content='I am sorry, I cannot fulfill this request. The available tools lack the desired functionality.', additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 32, 'candidates_token_count': 18, 'total_token_count': 50, 'prompt_tokens_details': [{'modality': 1, 'token_count': 32}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 18}], 'thoughts_token_count': 0, 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.01729025277826521, 'model_name': 'gemini-2.0-flash-lite-001'}, id='run--e48300c8-a3ed-4723-956b-2125e066fdc1-0', usage_metadata={'input_tokens': 32, 'output_tokens': 18, 'total_tokens': 50})

In [9]:
gemini_llm = init_chat_model(model="gemini-2.0-flash-lite-001", model_provider="google_vertexai")
from langgraph.prebuilt import ToolNode
toolnode = ToolNode([add, sub, mul, div])
gemini_llm_with_tools = gemini_llm.bind_tools([add, sub, mul, div])
def call_model(state: MessagesState) -> MessagesState:
    state['messages'] = gemini_llm_with_tools.invoke(state['messages'])
    return state

In [11]:
tool_graph_builder: StateGraph = StateGraph(MessagesState)
tool_graph_builder.add_node("llm", call_model)
tool_graph_builder.add_node("tools", toolnode)
tool_graph_builder.add_edge(START, "llm")
tool_graph_builder.add_edge("llm", "tools")
tool_graph_builder.add_edge("tools", END)

tools_graph = tool_graph_builder.compile()

In [13]:
result = tools_graph.invoke({ "messages": "What is 2 * 5 ?" })
result

{'messages': [HumanMessage(content='What is 2 * 5 ?', additional_kwargs={}, response_metadata={}, id='86c73852-b6bc-4758-9675-cb6ae87b84d6'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'multipy', 'arguments': '{"a": 2.0, "b": 5.0}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 65, 'candidates_token_count': 6, 'total_token_count': 71, 'prompt_tokens_details': [{'modality': 1, 'token_count': 65}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 6}], 'thoughts_token_count': 0, 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -9.904357527072231e-05, 'model_name': 'gemini-2.0-flash-lite-001'}, id='run--389eeee5-b60e-4d47-af92-ae568d610503-0', tool_calls=[{'name': 'multipy', 'args': {'a': 2.0, 'b': 5.0}, 'id': '458154c1-e311-4334-9e01-3ca075caf929', 'type': 'tool_call'}], usage_metadata={'input_tokens': 65, 'output_tokens': 6, 'total_tokens'

In [14]:
result = tools_graph.invoke({ "messages": "I have 5 pens, now i purchase 3 more. How many pens do i have ?" })
result

{'messages': [HumanMessage(content='I have 5 pens, now i purchase 3 more. How many pens do i have ?', additional_kwargs={}, response_metadata={}, id='fa762e62-f4c6-4623-b94d-dd3c03ba018a'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'addition', 'arguments': '{"a": 5.0, "b": 3.0}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 77, 'candidates_token_count': 5, 'total_token_count': 82, 'prompt_tokens_details': [{'modality': 1, 'token_count': 77}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 5}], 'thoughts_token_count': 0, 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.0003762520384043455, 'model_name': 'gemini-2.0-flash-lite-001'}, id='run--b019261a-72be-4dcf-9cbc-640259ebc32a-0', tool_calls=[{'name': 'addition', 'args': {'a': 5.0, 'b': 3.0}, 'id': '090ff30c-edcc-407f-9c56-14ca454e2785', 'type': 'tool_call'}], usage_metadata={'inp

In [15]:
result = tools_graph.invoke({ "messages": "I have 5 pens, now i lost 3 . How many pens do i have ?" })
result


{'messages': [HumanMessage(content='I have 5 pens, now i lost 3 . How many pens do i have ?', additional_kwargs={}, response_metadata={}, id='40adfb72-5484-47f9-9ffd-5104cc4466f5'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'subtraction', 'arguments': '{"a": 5.0, "b": 3.0}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 76, 'candidates_token_count': 6, 'total_token_count': 82, 'prompt_tokens_details': [{'modality': 1, 'token_count': 76}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 6}], 'thoughts_token_count': 0, 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.0819424440463384, 'model_name': 'gemini-2.0-flash-lite-001'}, id='run--0cb8589e-12d9-44cc-9eb1-1348309ec8bf-0', tool_calls=[{'name': 'subtraction', 'args': {'a': 5.0, 'b': 3.0}, 'id': 'df90de78-650c-428f-9f2e-972c3a4d5901', 'type': 'tool_call'}], usage_metadata={'input_to